In [1]:
import cellpylib
import pandas
import numpy
import os

In [2]:
path_test_kpi_raw = r"C:\Users\xedua\OneDrive\Escritorio\MCC-I\Research\Thesis_Confirmatory_Experiments\Instances\Test"
path_results = r"C:\Users\xedua\OneDrive\Escritorio\MCC-I\Research\Thesis_Confirmatory_Experiments\Results\Wolfram-CA_results"

In [3]:
# Generator/Selector of PIs
def instances(lib, instance_number, number_objects=100, difficulty="EASY"):
    fileName = path_test_kpi_raw + f"\\GA-{lib}_{difficulty}_{number_objects}_{instance_number:03d}.kp"
    with open(fileName, "r") as f:
        lines = f.readlines()
    line = lines[0].split(",")
    nbItems = int(line[0].strip())
    k_limit = int(line[1].strip())
    PI = [None] * nbItems
    for i in range(nbItems):
        line = lines[i + 1].split(",")
        weight = int(line[0].strip())
        profit = float(line[1].strip())
        PI[i] = (profit, weight)
    return PI, k_limit

In [4]:
def CA(rule, seed = None, number_objects=100):
    if seed == None:
        init = cellpylib.init_random(number_objects)
    else:
        init = numpy.array([seed])
    cellular_automaton = cellpylib.evolve(init,
                                          timesteps = number_objects,
                                          memoize = True,
                                          apply_rule = lambda 
                                          n,
                                          c, 
                                          t: cellpylib.nks_rule(n, rule))
    return cellular_automaton

In [5]:
# Maximum Profit per Weight Unit Solution Generator
def MAPW(PI, k_limit):
    number_objects = len(PI)
    wgt = 0
    MAPW = []
    solution = [0] * number_objects # Creates solution template of 0s
    for i in range(number_objects):
        MAPW.append((i, PI[i][0], PI[i][1], PI[i][0]/PI[i][1])) # Adds a new 
        # indexed problems list
    MAPW = sorted(MAPW, reverse = True, key = lambda x: x[3]) # Sorts the new list
    # by weight
    for object in MAPW:
        if object[2] > k_limit:
            continue
        wgt += object[2] # Evaluates the acumulated weight
        if wgt <= k_limit: # When the knapsack is broken, omits the object
            solution[object[0]] = 1 # Adds the most profitable object until full
        else:
            wgt -= object[2]
    return solution

In [6]:
# Minimum Weight Heuristic Solution Generator
def MIW(PI, k_limit):
    number_objects = len(PI)
    wgt = 0
    MIW = []
    solution = [0] * number_objects # Creates solution template of 0s
    for i in range(number_objects):
        MIW.append((i, PI[i][0], PI[i][1])) # Adds a new indexed problems list
    MIW = sorted(MIW, key = lambda x: x[2]) # Sorts the new list by weight
    for object in MIW:
        if object[2] > k_limit:
            continue
        wgt += object[2] # Evaluates the acumulated weight
        if wgt <= k_limit: # When the knapsack is broken, omits the object
            solution[object[0]] = 1 # Adds the lighter object until full
        else:
            wgt -= object[2]
    return solution

In [7]:
# Evaluator of Solutions
def evaluator(PI, solution, k_limit):
    s1 = (0, 0, 0)
    for i in range(len(PI)): # Iterates all the objects in the problem instance
        if solution[i] == 1: # When the object is in the knapsack considers the 
            # object in the evaluation
            s1 = (k_limit, s1[1] + PI[i][0], s1[2] + PI[i][1]) # Sums up the profit and 
            # the weight of all the items
    if s1[2] <= k_limit: # When the knapsack is not broken saves a record of 0
        s1 = (k_limit, s1[1], s1[2], 0, solution)
    else:
        s1 = (k_limit, s1[1], s1[2], 1, solution) # When the knapsack is broken saves a record of 1
    return s1

In [8]:
def SolutionAssembler(PI, k_limit, strategy):
    wgt = 0
    solution = [0] * len(PI)
    solvers = [MIW, MAPW]
    for i in range(len(strategy)):
        if PI[i][1] > k_limit:
            continue
        result = solvers[strategy[i]](PI[i:], k_limit)[0]
        if result == 1:
            wgt += PI[i][1]
            if wgt > k_limit:
                wgt -= PI[i][1]
                solution[i] = 0 
            else:
                solution[i] = result
    return solution

In [9]:
# Cellular Automata Solution Generator (PBMH)
def PBMH_SolutionAssembler(PI, k_limit, strategy):
    wgt = 0
    solution = [0] * len(PI)
    solvers = [MIW, MAPW]
    for i in range(len(strategy)):
        if PI[i][1] > k_limit:
            continue
        result = solvers[strategy[i]](PI, k_limit)        
        if result[i] == 1:
            wgt += PI[i][1]
            if wgt > k_limit:
                wgt -= PI[i][1]
                solution[i] = 0 
            else:
                solution[i] = result[i]
    return solution

In [10]:
def seeds_generator(heuristic_index, number_objects=100, difficulty="EASY"):
    libs = ['DEF', 'MAXPW', 'MAXP', 'MINW']
    PI_nums = [num for num in range(number_objects)]
    PI_labels = [f'GA-{lib}_{difficulty}_100_{str(num).zfill(3)}' for lib in libs for num in range(number_objects)]
    evaluations = []
    for lib in libs:
        for PI_num in PI_nums:
            PI, k_limit = instances(lib, PI_num)
            solvers = [MIW(PI, k_limit), MAPW(PI, k_limit)]
            solution = solvers[heuristic_index]
            evaluations.append(evaluator(PI, solution, k_limit))
    df = pandas.DataFrame(evaluations, columns = ['Knapsack Limit', 'Profit', 'Weight', 'Knapsack State', 'Solution'])
    df.index = PI_labels
    df.index.names = ['Problem Instance']
    return df

In [11]:
def solver(heuristics, HH_method, number_objects = 100):
    libs = ['DEF', 'MAXPW', 'MAXP', 'MINW']
    PI_nums = [num for num in range(number_objects)]
    results = {}
    for rule in range(1, 256):
        evaluations = []
        i = 0
        for lib in libs:
            for PI_num in PI_nums:
                PI, k_limit = instances(lib, PI_num)                
                solution = HH_method(PI, k_limit, heuristics[str(rule)][i])
                evaluations.append(evaluator(PI, solution, k_limit))
                i += 1                    
        results[rule] = evaluations            
    return results

In [12]:
def strategy_gen():
    seeds = seeds_generator(1).Solution
    heuristics = {}
    for rule in range(1, 256):
        strategies = []
        for seed in seeds:
            strategies.append(CA(rule, seed)[-1])
        heuristics[str(rule)] = strategies
    return heuristics

In [13]:
def run_experiment(strategy_gen, solver, seeds_generator, PI_labels, path_results, experiment_name, HH_method):
    heuristics = strategy_gen()
    results = solver(heuristics, HH_method)
    profits = {key: [evaluation[1] for evaluation in value] for key, value in results.items()}
    sum_profits = {key: sum(values) for key, values in profits.items()}
    best_heuristic_rname = max(sum_profits, key=sum_profits.get)
    best_performance_score = sum_profits[best_heuristic_rname]
    best_heuristics = heuristics[str(best_heuristic_rname)]
    best_results = results[best_heuristic_rname]

    df = pandas.DataFrame(best_results, columns=["Knapsack Limit", "Profit", "Weight", "Knapsack State", "Solution"])
    df['Strategy'] = best_heuristics
    df.index = PI_labels
    df.index.names = ['Problem Instance']
    results_file = os.path.join(path_results, f'{experiment_name}_Results.csv')
    df.to_csv(results_file, encoding='utf-8')

    print(f"Data has been successfully written to {results_file}")
    print(f'Best CA heuristic: {best_heuristic_rname}, Performance: {best_performance_score}')

    MIW_results = seeds_generator(0).Profit
    MAXPW_results = seeds_generator(1).Profit
    print('MIW Performance:' + str(sum(MIW_results)), 'MAXPW Performance:' + str(sum(MAXPW_results)))

In [14]:
libs = ['DEF', 'MAXPW', 'MAXP', 'MINW']
PI_labels = [f'GA-{lib}_EASY_100_{str(num).zfill(3)}' for lib in libs for num in range(100)]

In [15]:
# Experiment with Conventional Selection HH
run_experiment(strategy_gen, solver, seeds_generator, PI_labels, path_results, "CAHH-Wolfram", SolutionAssembler)

Data has been successfully written to C:\Users\xedua\OneDrive\Escritorio\MCC-I\Research\Thesis_Confirmatory_Experiments\Results\Wolfram-CA_results\CAHH-Wolfram_Results.csv
Best CA heuristic: 235, Performance: 330551.0
MIW Performance:271660.0 MAXPW Performance:356612.0


In [16]:
# Experiment with PBMH Based Selection HH
run_experiment(strategy_gen, solver, seeds_generator, PI_labels, path_results, "CAHH-Wolfram-PBMH", PBMH_SolutionAssembler)

Data has been successfully written to C:\Users\xedua\OneDrive\Escritorio\MCC-I\Research\Thesis_Confirmatory_Experiments\Results\Wolfram-CA_results\CAHH-Wolfram-PBMH_Results.csv
Best CA heuristic: 235, Performance: 356612.0
MIW Performance:271660.0 MAXPW Performance:356612.0
